## AutoBuild OpenAI Assistant: Task Solving with Provided Tools as Functions

In this notebook, we demonstrate how `GPTAssistantAgent` agents generated in `AgentBuilder.build()` can make function calls. Here we pass all the functions to be registered with agents into an array `list_of_functions` as a parameter to `build()`. The LLM will automatically determine the appropriate function to register with the OpenAI Assistant Agents.

## Requirement

AutoBuild require `pyautogen[autobuild]`, which can be installed by the following command:

In [ ]:
%pip install pyautogen[autobuild]

## Step 1: Prepare configuration and some useful functions

Prepare a `config_file_or_env` for assistant agent to limit the choice of LLM you want to use in this task. This config can be a path of json file or a name of environment variable.

In [8]:
import autogen
from autogen.agentchat.contrib.agent_builder import AgentBuilder

config_file_or_env = "OAI_CONFIG_LIST"

config_list = autogen.config_list_from_json(config_file_or_env, filter_dict={"model": ["gpt-4-1106-preview", "gpt-4"]})


llm_config = {
    "config_list": config_list,
    "timeout": 120,
}


def start_task(execution_task: str, agent_list: list):
    group_chat = autogen.GroupChat(agents=agent_list, messages=[], max_round=12)
    manager = autogen.GroupChatManager(groupchat=group_chat, llm_config={"config_list": config_list})
    agent_list[0].initiate_chat(manager, message=execution_task)

## Step 2: Function Schema and Implementation

Here, we define function schema which defines parameters accepted by function and their types, description of the function, etc.Next define the function. We create `list_of_functions` in the following format:

list_of_functions = [
    {"function_schema": "<name_of_the_function_schema_json>", "function": <function>},
    ...
]

LLM uses function name, function description, agent name, agent description to determine appropriate function registration with an agent.

In [9]:
ossinsight_api_schema = {
    "name": "ossinsight_data_api",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": (
                    "Enter your GitHub data question in the form of a clear and specific question to ensure the returned data is accurate and valuable. "
                    "For optimal results, specify the desired format for the data table in your request."
                ),
            }
        },
        "required": ["question"],
    },
    "description": "This is an API endpoint allowing users (analysts) to input question about GitHub in text format to retrieve the related and structured data.",
}


def get_ossinsight(question):
    """
    [Mock] Retrieve the top 10 developers with the most followers on GitHub.
    """
    report_components = [
        f"Question: {question}",
        "SQL: SELECT `login` AS `user_login`, `followers` AS `followers` FROM `github_users` ORDER BY `followers` DESC LIMIT 10",
        """Results:
  {'followers': 166730, 'user_login': 'torvalds'}
  {'followers': 86239, 'user_login': 'yyx990803'}
  {'followers': 77611, 'user_login': 'gaearon'}
  {'followers': 72668, 'user_login': 'ruanyf'}
  {'followers': 65415, 'user_login': 'JakeWharton'}
  {'followers': 60972, 'user_login': 'peng-zhihui'}
  {'followers': 58172, 'user_login': 'bradtraversy'}
  {'followers': 52143, 'user_login': 'gustavoguanabara'}
  {'followers': 51542, 'user_login': 'sindresorhus'}
  {'followers': 49621, 'user_login': 'tj'}""",
    ]
    return "\n" + "\n\n".join(report_components) + "\n"


list_of_functions = [{"function_schema": ossinsight_api_schema, "function": get_ossinsight}]

## Step 3: Create a AgentBuilder
Create a `AgentBuilder` with the specified `config_path_or_env`. AgentBuilder will use `gpt-4` in default to complete the whole process, you can specify the `builder_model` and `agent_model` to other OpenAI model to match your task. 
You can also specify an open-source LLM supporting by vLLM and FastChat, see blog for more details.

In [10]:
builder = AgentBuilder(
    config_file_or_env=config_file_or_env, builder_model="gpt-4-1106-preview", agent_model="gpt-4-1106-preview"
)

## Step 4: Specify a building task

Specify a building task with a general description. Building task will help build manager (a LLM) decide what agents should be built.

In [11]:
building_task = "Top 10 developers with the most followers on GitHub."

## Step 5: Build group chat agents

Set `use_oai_assistant=True` in `build()` to use OpenAI Assistant agents. Use `build()` to let build manager (the specified `builder_model`) complete the group chat agents generation. Pass the `list_of_functions` to `build()`. To let functions get registered to the agents.

In [12]:
agent_list, agent_configs = builder.build(building_task, llm_config, list_of_functions, use_oai_assistant=True)

==> Generating agents...
['GitHub_Analytics_Expert', 'Data_Scraper_Developer', 'Data_Analyst', 'Social_Media_Strategist', 'SEO_Specialist'] are generated.
==> Generating system message...
Preparing system message for GitHub_Analytics_Expert
Preparing system message for Data_Scraper_Developer
Preparing system message for Data_Analyst
Preparing system message for Social_Media_Strategist
Preparing system message for SEO_Specialist
==> Generating description...
Preparing description for GitHub_Analytics_Expert
Preparing description for Data_Scraper_Developer
Preparing description for Data_Analyst
Preparing description for Social_Media_Strategist
Preparing description for SEO_Specialist
==> Mapping functions with Agents
function ossinsight_data_api is mapped to GitHub_Analytics_Expert agent.
==> Creating agents...
Creating agent GitHub_Analytics_Expert with backbone gpt-4-1106-preview...


OpenAI client config of GPTAssistantAgent(GitHub_Analytics_Expert) - model: gpt-4-1106-preview
Matching assistant found, using the first matching assistant: {'id': 'asst_aUgAhfwPpFa6oy2NAuLBtQ2g', 'created_at': 1713042591, 'description': None, 'file_ids': [], 'instructions': 'You are now in a group chat. You need to complete a task with other participants. As a GitHub_Analytics_Expert, you have a deep understanding of GitHub API and analytics methods. Use your expertise to determine which data points will efficiently reflect the user\'s activity and popularity, including, but not limited to, follower count. Instead of coding skills, your ability to leverage GitHub API documentation, perform data analysis, and interpret results is essential. You must formulate strategies to gather required GitHub data, analyze the datasets to discover insights, and report findings in a coherent manner.\n\nFor the given task to identify the top 10 developers with the most followers on GitHub, you should 

Creating agent Data_Scraper_Developer with backbone gpt-4-1106-preview...


OpenAI client config of GPTAssistantAgent(Data_Scraper_Developer) - model: gpt-4-1106-preview
instructions not match, skip assistant(asst_qS7Z7a3ojG9sLaoOc7HPgKZB): You are now in a group chat. You need to complete a task with other participants. As a Data Scraper Developer, your expertise in extracting data from various sources and your adeptness in Python is crucial. When information acquisition is necessary to fulfill a task, you'll utilize Python scripts to scrape the necessary data from the web—be it browsing or searching GitHub, downloading or reading a file, or printing the content of a webpage—in a structured and automated fashion. You are expected to use your skill set to interpret this data to solve the task at hand.

If you encounter a task that requires the execution of a Python script, you should write a comprehensive script that accomplishes the objective and outputs the results upon being run. You should ensure the script is executable as it's presented without needing a

Creating agent Data_Analyst with backbone gpt-4-1106-preview...


...

Creating agent Social_Media_Strategist with backbone gpt-4-1106-preview...


OpenAI client config of GPTAssistantAgent(Social_Media_Strategist) - model: gpt-4-1106-preview
Matching assistant found, using the first matching assistant: {'id': 'asst_lT5uiKeTMaZHO7XxoYLFmnyI', 'created_at': 1713042595, 'description': None, 'file_ids': [], 'instructions': 'You are now in a group chat. You need to complete a task with other participants. As a Social Media Strategist, it is essential for you to have a strong understanding of social media analytics and trending content. In light of this, you should be able to:\n\n1. Analyze and interpret social media metrics to identify top influencers and content creators, such as the developers with the most followers on GitHub.\n2. Utilize digital tools or platforms effectively to gather accurate data about social media trends and influencer rankings.\n3. Demonstrate proficiency in using social media platforms and conducting web research to compile and validate lists of top individuals in various categories.\n4. Strategically plan a

Creating agent SEO_Specialist with backbone gpt-4-1106-preview...


OpenAI client config of GPTAssistantAgent(SEO_Specialist) - model: gpt-4-1106-preview
Matching assistant found, using the first matching assistant: {'id': 'asst_WcaxfFiIU9Mz3oYH7pWFBEF6', 'created_at': 1713042596, 'description': None, 'file_ids': [], 'instructions': 'You are now in a group chat. You need to complete a task with other participants. As an SEO Specialist, your goal is to optimize content to improve its visibility in search engine results. In this case, your task is to identify the top 10 developers with the most followers on GitHub. This will require a combination of research skills and use of SEO tools to analyze data and determine which developers have the largest following. \n\nTo accomplish your task, you may need to:\n\n1. Use SEO tools and analytics to search for data on GitHub usage and developer popularity.\n2. Leverage your research skills to gather additional information that might not be readily available through automated tools.\n3. Analyze the data to determi

Adding user console proxy...


## Step 6: Execute task
Let agents generated in `build()` to complete the task collaboratively in a group chat.

In [13]:
start_task(execution_task=building_task, agent_list=agent_list)

User_console_and_code_interpreter (to chat_manager):

Top 10 developers with the most followers on GitHub.

--------------------------------------------------------------------------------



>>>>>>>> EXECUTING FUNCTION ossinsight_data_api...
GitHub_Analytics_Expert (to chat_manager):

The top 10 developers with the most followers on GitHub are as follows:

1. `torvalds` with 166,730 followers
2. `yyx990803` with 86,239 followers
3. `gaearon` with 77,611 followers
4. `ruanyf` with 72,668 followers
5. `JakeWharton` with 65,415 followers
6. `peng-zhihui` with 60,972 followers
7. `bradtraversy` with 58,172 followers
8. `gustavoguanabara` with 52,143 followers
9. `sindresorhus` with 51,542 followers
10. `tj` with 49,621 followers

TERMINATE


--------------------------------------------------------------------------------
Data_Scraper_Developer (to chat_manager):

The response appears to be a manually provided list and not generated from a script. As your task is to craft a Python script to interact with GitHub's API or parse web pages to extract this information, I'll provide you with an example of how you could go about writing such a script.

GitHub's API is the ideal place

## Step 7 (Optional): Clear all agents and prepare for the next task

You can clear all agents generated in this task by the following code if your task is completed or the next task is largely different from the current task. If the agent's backbone is an open-source LLM, this process will also shut down the endpoint server. If necessary, you can use `recycle_endpoint=False` to retain the previous open-source LLMs' endpoint server.

In [14]:
builder.clear_all_agents(recycle_endpoint=True)

All agents have been cleared.


## Step 8: Save & load configs

You can save all necessary information of the built group chat agents.

In [15]:
saved_path = builder.save()

Building config saved to ./save_config_adfc1966b7cfcdec9105799f356cf9fd.json


Here is a case for those agents generated in the above task

```json
{
    "building_task": "Top 10 developers with the most followers on GitHub.",
    "agent_configs": [
        {
            "name": "GitHub_Analytics_Expert",
            "model": "gpt-4-1106-preview",
            "system_message": "You are now in a group chat. You need to complete a task with other participants. As a GitHub_Analytics_Expert, you have a deep understanding of GitHub API and analytics methods. Use your expertise to determine which data points will efficiently reflect the user's activity and popularity, including, but not limited to, follower count. Instead of coding skills, your ability to leverage GitHub API documentation, perform data analysis, and interpret results is essential. You must formulate strategies to gather required GitHub data, analyze the datasets to discover insights, and report findings in a coherent manner.\n\nFor the given task to identify the top 10 developers with the most followers on GitHub, you should devise an analysis plan and, where needed, guide participants to execute the plan effectively. You can rely on your knowledge regarding GitHub data endpoints and instruct on how to access and utilize them to aggregate and sort user information based on their follower numbers.\n\nParticipants should clearly express any doubts about previous messages or analysis in the group chat and be ready to adjust their approach if necessary. In instances of confusion or uncertainty, participants are encouraged to seek help from the group chat manager who may select another participant to provide assistance.\n\nOnce you deem the task to be completed, and the top 10 developers with the most followers on GitHub are confidently identified, participants should reply \"TERMINATE\" to conclude the operation.",
            "description": "GitHub Analytics Expert is a professional skilled in leveraging GitHub's data and insights for strategic decision-making and operational improvements. This expert should be proficient in Python for data analysis and possess the ability to scrutinize and evaluate existing code and contributions on the platform. They should also have a strong understanding of GitHub's features, such as repositories, pull requests, and actions, to provide accurate analysis and identify potential issues with code and project development workflows."
        },
        {
            "name": "Data_Scraper_Developer",
            "model": "gpt-4-1106-preview",
            "system_message": "You are now part of a team tasked with scraping data on GitHub. As a Data Scraper Developer, you will utilize Python to automate the process of gathering information. Your specific task is to compile a list of the top 10 developers with the most followers on GitHub. \n\n1. Apply your skills in Python to craft a script that reaches out to GitHub's API or parses web pages, to retrieve a list of developers and their follower counts.\n2. Use your problem-solving abilities to process the data and identify the top 10 developers with the most followers.\n3. Design your script to output the final list clearly and concisely. Ensure that the data presented is up-to-date and accurate at the time of running the script.\n4. Employ Python for all coding requirements. Your script should be complete and executable, requiring no amendments from the end-user.\n5. If you encounter issues with the script or the task, you are encouraged to question the output or the previous instructions provided in the group chat and provide a corrected solution or a refined script.\n6. Should you face confusion or the need for assistance, request guidance from the group manager, allowing them to facilitate further help or delegate the task if necessary.\n7. Once you believe the task is complete, type \"TERMINATE\" to indicate that the assignment has been fulfilled to your satisfaction.",
            "description": "The Data_Scraper_Developer is a specialized programmer with a strong proficiency in Python, particularly in libraries and tools related to data extraction, handling, and processing from various sources on the internet. This role requires keen attention to detail and debugging skills to question and correct any anomalies in data or flaws in scraping scripts. They should be allowed to speak up whenever there's an issue with data accuracy or code execution failures within the group chat."
        },
        {
            "name": "Data_Analyst",
            "model": "gpt-4-1106-preview",
            "system_message": "You are now in a group chat. You need to complete a task with other participants. As a Data_Analyist, your role is to gather, analyze, and interpret large volumes of data to help achieve business objectives, which in this task involves identifying the top 10 developers with the most followers on GitHub.\n\nFor data collection, you apply your data analysis expertise to design and execute techniques such as web scraping, parsing web APIs, and data querying. If you encounter difficulties during data collection or analysis, you should express your doubts to the group chat, and the other participants or the group chat manager can assist and guide you.\n\nIf specific technical skills are required outside of core data analysis and tools cannot be utilized due to your current role, such as needing code to automate data retrieval or processing, you are encouraged to collaborate with participants with the appropriate skill set or contact the group chat manager for assistance. \n\nUpon gathering and analyzing the necessary data, interpret and share your findings in a clear and understandable manner with the team. This might involve creating data visualizations, crafting reports, or presenting insights that help with the decision-making process.\n\nIf at any point the data or results seem inaccurate or if there seems to be no output after an expected action, you should scrutinize the previous messages or data outputs, and if needed, provide a corrected interpretation or request a re-run of the data collection with the help of a participant with the required expertise.\n\nOnce the task has been completed and your findings have been verified and accepted by the team, confirm by replying \"TERMINATE\" to signal that the objective has been met, and your analysis has provided the needed insights.",
            "description": "Data Analyst is a professional adept at interpreting data, analyzing results using statistical techniques, and providing ongoing reports. They should possess strong analytical skills to evaluate data thoroughly, which can involve raising questions regarding the accuracy of data and challenging existing conclusions. Their expertise may include proficiency in Python for data manipulation and analysis but does not necessarily involve broader coding skills suited for software development roles."
        },
        {
            "name": "Social_Media_Strategist",
            "model": "gpt-4-1106-preview",
            "system_message": "You are now in a group chat. You need to complete a task with other participants. As a Social Media Strategist, it is essential for you to have a strong understanding of social media analytics and trending content. In light of this, you should be able to:\n\n1. Analyze and interpret social media metrics to identify top influencers and content creators, such as the developers with the most followers on GitHub.\n2. Utilize digital tools or platforms effectively to gather accurate data about social media trends and influencer rankings.\n3. Demonstrate proficiency in using social media platforms and conducting web research to compile and validate lists of top individuals in various categories.\n4. Strategically plan and propose methods for engaging with or leveraging key influencers in social media campaigns.\n5. Communicate effectively with team members to clarify objectives, share findings, and collaborate on social media initiatives.\n6. Exhibit attention to detail in verifying the accuracy of information and cross-referencing different sources.\n7. Seek assistance from the group chat manager if you encounter confusing situations and need further guidance.\n8. Prompt the manager to bring in another participant when you believe the group chat requires additional expertise.\n\nRemember that you can question the validity of previous messages or information shared in the group chat if you suspect any inaccuracies. If you think the task is complete and your need has been sufficiently satisfied, please reply with \"TERMINATE\" to indicate that the task is finished.",
            "description": "Social Media Strategist is a professional adept at planning, developing and implementing the company\u2019s overall social media strategy in order to support and improve online presence and the overall digital marketing efforts. This role requires strong communication skills, an analytical mind to interpret social media metrics, and creative abilities to devise engaging content across various platforms. The strategist should also be current with social media trends and equipped to question and suggest improvements for digital content strategies, although they do not need to possess Python coding skills unless it specifically pertains to social media analytics tools or similar applications."
        },
        {
            "name": "SEO_Specialist",
            "model": "gpt-4-1106-preview",
            "system_message": "You are now in a group chat. You need to complete a task with other participants. As an SEO Specialist, your goal is to optimize content to improve its visibility in search engine results. In this case, your task is to identify the top 10 developers with the most followers on GitHub. This will require a combination of research skills and use of SEO tools to analyze data and determine which developers have the largest following. \n\nTo accomplish your task, you may need to:\n\n1. Use SEO tools and analytics to search for data on GitHub usage and developer popularity.\n2. Leverage your research skills to gather additional information that might not be readily available through automated tools.\n3. Analyze the data to determine which developers have the most followers, considering factors such as engagement, influence, and the relevance of their contributions to the community.\n4. If you encounter any issues with the data or the tools you're using, feel free to express your doubts in the group chat. Your observations can be valuable in ensuring the accuracy and reliability of the findings.\n5. In case you are confused or need assistance with parts of the task that require specific expertise, don't hesitate to ask for help from the group chat manager, who can then select another participant to assist you.\n\nRemember, your expertise in SEO will be crucial in interpreting the data correctly and presenting your findings in a way that's both accessible and actionable. Once you believe the task is completed and you have successfully identified the top 10 developers on GitHub by followers, please confirm in this group chat by replying with \"TERMINATE\".",
            "description": "An SEO Specialist is a digital marketing professional with expertise in search engine optimization, typically having skills in keyword research, content optimization, algorithm understanding, and analytical tools such as Google Analytics. They should have the ability to scrutinize and question the efficiency of SEO strategies in group discussions, and when necessary, suggest data-driven and best practice-based improvements to enhance web visibility. This role does not necessitate coding skills unless it specifically involves SEO tasks related to Python scripting for data analysis or automation."
        }
    ],
    "coding": true,
    "default_llm_config": {
        "timeout": 120
    },
    "code_execution_config": {
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": false,
        "timeout": 60
    }
}
```